In [61]:
import pandas as pd
import numpy as np
from string import punctuation
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Embedding, Dense, LSTM

In [17]:
df = pd.read_csv('ArticlesApril2018.csv')
df.head()

,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,5adf6684068401528a2aa69b,781,By JOHN BRANCH,article,Former N.F.L. Cheerleaders’ Settlement Offer: ...,"['Workplace Hazards and Violations', 'Football...",68,Sports,0,2018-04-24 17:16:49,Pro Football,"“I understand that they could meet with us, pa...",The New York Times,News,https://www.nytimes.com/2018/04/24/sports/foot...
1,5adf653f068401528a2aa697,656,By LISA FRIEDMAN,article,E.P.A. to Unveil a New Rule. Its Effect: Less ...,"['Environmental Protection Agency', 'Pruitt, S...",68,Climate,0,2018-04-24 17:11:21,Unknown,The agency plans to publish a new regulation T...,The New York Times,News,https://www.nytimes.com/2018/04/24/climate/epa...
2,5adf4626068401528a2aa628,2427,By PETE WELLS,article,"The New Noma, Explained","['Restaurants', 'Noma (Copenhagen, Restaurant)...",66,Dining,0,2018-04-24 14:58:44,Unknown,What’s it like to eat at the second incarnatio...,The New York Times,News,https://www.nytimes.com/2018/04/24/dining/noma...
3,5adf40d2068401528a2aa619,626,By JULIE HIRSCHFELD DAVIS and PETER BAKER,article,Unknown,"['Macron, Emmanuel (1977- )', 'Trump, Donald J...",68,Washington,0,2018-04-24 14:35:57,Europe,President Trump welcomed President Emmanuel Ma...,The New York Times,News,https://www.nytimes.com/2018/04/24/world/europ...
4,5adf3d64068401528a2aa60f,815,By IAN AUSTEN and DAN BILEFSKY,article,Unknown,"['Toronto, Ontario, Attack (April, 2018)', 'Mu...",68,Foreign,0,2018-04-24 14:21:21,Canada,"Alek Minassian, 25, a resident of Toronto’s Ri...",The New York Times,News,https://www.nytimes.com/2018/04/24/world/canad...


In [18]:
print(df.columns, len(df.columns))

Index(['articleID', 'articleWordCount', 'byline', 'documentType', 'headline',
       'keywords', 'multimedia', 'newDesk', 'printPage', 'pubDate',
       'sectionName', 'snippet', 'source', 'typeOfMaterial', 'webURL'],
      dtype='object') 15


In [19]:
print(df['headline'].isnull().values.any())

False


In [20]:
headline = []
headline.extend(list(df.headline.values))
headline[:5]

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'Unknown',
 'Unknown']

In [21]:
print('총 샘플 개수 : ', len(headline))

총 샘플 개수 :  1324


In [22]:
headline = [word for word in headline if word != 'Unkown']
print('총 샘플 개수 :', len(headline))

총 샘플 개수 : 1324


In [23]:
headline[:5]

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'Unknown',
 'Unknown']

In [24]:
def repreprocessing(raw_sentence):
    # utf-8로 인코딩 된 문자열을 ASCII 문자열로 변환한다.(표현할 수 없는 문자는 무시)
    sentence = raw_sentence.encode('utf-8').decode('ascii', 'ignore')
    return ''.join(word for word in sentence if word not in punctuation).lower()

In [25]:
preprocessed_headline = [repreprocessing(x) for x in headline]
preprocessed_headline[:5]

['former nfl cheerleaders settlement offer 1 and a meeting with goodell',
 'epa to unveil a new rule its effect less science in policymaking',
 'the new noma explained',
 'unknown',
 'unknown']

In [26]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(preprocessed_headline)
vocab_size = len(tokenizer.word_index) + 1
print('사전 크기 : ', vocab_size)

사전 크기 :  3495


In [29]:
sequences = list()
for sentence in preprocessed_headline:
    encoded = tokenizer.texts_to_sequences([sentence])[0]
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)

sequences[:11]

[[100, 270],
 [100, 270, 372],
 [100, 270, 372, 1116],
 [100, 270, 372, 1116, 583],
 [100, 270, 372, 1116, 583, 53],
 [100, 270, 372, 1116, 583, 53, 7],
 [100, 270, 372, 1116, 583, 53, 7, 2],
 [100, 270, 372, 1116, 583, 53, 7, 2, 373],
 [100, 270, 372, 1116, 583, 53, 7, 2, 373, 11],
 [100, 270, 372, 1116, 583, 53, 7, 2, 373, 11, 1117],
 [101, 3]]

In [35]:
index_to_word = {}
for key, value in tokenizer.word_index.items():
    index_to_word[value] = key

print('빈도수 기준 단어 : ', index_to_word[99])

빈도수 기준 단어 :  border


In [53]:
# 시퀸스에 대한 패딩작업(sequences) -> X, y 분리 
max_length = max(len(seq) for seq in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='pre')

print('샘플 최대 길이 : ', max_length)
print(sequences[:3])

샘플 최대 길이 :  24
[[100, 270], [100, 270, 372], [100, 270, 372, 1116]]


In [54]:
# X, y 분리
X = padded_sequences[:, :-1]
y = padded_sequences[:, -1]

In [55]:
print(X)

[[   0    0    0 ...    0    0  100]
 [   0    0    0 ...    0  100  270]
 [   0    0    0 ...  100  270  372]
 ...
 [   0    0    0 ...  171    9 3494]
 [   0    0    0 ...    9 3494  116]
 [   0    0    0 ... 3494  116    2]]


In [56]:
print(y)

[ 270  372 1116 ...  116    2 1026]


In [57]:
y = to_categorical(y, num_classes=vocab_size)
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [62]:
embedding_dim = 10
hidden_units = 128

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(LSTM(hidden_units))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=200, verbose=2)

Epoch 1/200
244/244 - 6s - loss: 7.6280 - accuracy: 0.0263 - 6s/epoch - 25ms/step
Epoch 2/200
244/244 - 6s - loss: 7.1000 - accuracy: 0.0290 - 6s/epoch - 23ms/step
Epoch 3/200
244/244 - 5s - loss: 6.9724 - accuracy: 0.0342 - 5s/epoch - 22ms/step
Epoch 4/200
244/244 - 6s - loss: 6.8478 - accuracy: 0.0396 - 6s/epoch - 24ms/step
Epoch 5/200
244/244 - 5s - loss: 6.6998 - accuracy: 0.0470 - 5s/epoch - 22ms/step
Epoch 6/200
244/244 - 6s - loss: 6.5335 - accuracy: 0.0502 - 6s/epoch - 24ms/step
Epoch 7/200
244/244 - 5s - loss: 6.3418 - accuracy: 0.0519 - 5s/epoch - 22ms/step
Epoch 8/200
244/244 - 5s - loss: 6.1355 - accuracy: 0.0573 - 5s/epoch - 22ms/step
Epoch 9/200
244/244 - 6s - loss: 5.9369 - accuracy: 0.0618 - 6s/epoch - 23ms/step
Epoch 10/200
244/244 - 7s - loss: 5.7469 - accuracy: 0.0637 - 7s/epoch - 27ms/step
Epoch 11/200
244/244 - 10s - loss: 5.5694 - accuracy: 0.0675 - 10s/epoch - 40ms/step
Epoch 12/200
244/244 - 6s - loss: 5.3988 - accuracy: 0.0751 - 6s/epoch - 23ms/step
Epoch 13/20

In [74]:
def sentence_generation(model, tokenizer, current_word, n):
    # 초기값
    init_word = current_word
    sentence = ''

    # n번의 단어 생성
    for _ in range(n):
        # 입력 단어에 대한 정수 인코딩
        encoded = tokenizer.texts_to_sequences([current_word])[0]
        encoded = pad_sequences([encoded], maxlength=max_length-1, padding='pre')
        
        # 단어 예측
        result = model.predict(encoded, verbose=0)
        result = np.argmax(result, axis=1)
        predicted_index = mpdel.predict(encoded, verbose=0).argmax(axis=-1)[0]

        # 인덱스에 맞는 단어 찾기
        for word, index in tokenizer.word_index.items():
            if index == result:
                break

        # 문장에 단어 추가
        current_word = current_word + ' ' + word
        sentence = sentence + ' ' + word

    sentence = init_word + sentence
    return init_word + sentence

In [75]:
sentence_generation(model, tokenizer, 'former', 9)

TypeError: pad_sequences() got an unexpected keyword argument 'maxlength'

In [76]:
sentence_generation(model, tokenizer, 'i', 10)

TypeError: pad_sequences() got an unexpected keyword argument 'maxlength'

In [ ]:
sentence_generation(model, tokenizer, 'how', 5)

# = 뉴스 헤더라인 생성기